In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

# Paths to ZIP files in Drive
zip1 = "/content/drive/MyDrive/all_videos1.zip"
zip2 = "/content/drive/MyDrive/all_videos2.zip"

# Where to extract them
dest1 = "/content/frames_zip1"
dest2 = "/content/frames_zip2"

# Extract
for zip_path, dest in [(zip1, dest1), (zip2, dest2)]:
    os.makedirs(dest, exist_ok=True)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dest)

print("✅ Both ZIPs extracted.")

✅ Both ZIPs extracted.


In [ ]:
import os

# Check contents of the first ZIP extraction
folders_zip1 = os.listdir("/content/frames_zip1")
folders_zip2 = os.listdir("/content/frames_zip2")

print("📁 ZIP 1 folders:", folders_zip1[:5])
print("📁 ZIP 2 folders:", folders_zip2[:5])


📁 ZIP 1 folders: ['all_videos_frames']
📁 ZIP 2 folders: ['all_videos_frames']


In [ ]:
import os

nested_root = "/content/frames_zip1/all_videos_frames"
folders = os.listdir(nested_root)

print("Found folders:", folders[:5])


Found folders: ['7454586412076158216_frames', '7442967118838009095_frames', '7441278772633980215_frames', '7445765705682701576_frames', '7453082732021812487_frames']


In [ ]:
sample_folder = os.path.join(nested_root, folders[0])
images = os.listdir(sample_folder)

print(f"Found {len(images)} images in {folders[0]}")
print("Examples:", images[:5])


🖼 Found 11 images in 7454586412076158216_frames
Examples: ['7454586412076158216_frame_0001.jpg', '7454586412076158216_frame_0004.jpg', '7454586412076158216_frame_0000.jpg', '7454586412076158216_frame_0002.jpg', '7454586412076158216_frame_0007.jpg']


In [ ]:
# These are the paths inside the unzipped folders
frame_sources = [
    "/content/frames_zip1/all_videos_frames",
    "/content/frames_zip2/all_videos_frames"
]

# Where to save output
output_dir = "/content/drive/MyDrive/clip_embeddings_all"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import torch
import os
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
def extract_clip_embedding(image_path):
    image = Image.open(image_path).convert("RGB")
    inputs = clip_processor(images=image, return_tensors="pt").to(device)
    with torch.no_grad():
        embedding = clip_model.get_image_features(**inputs)
    return embedding.squeeze().cpu().numpy()

In [ ]:
# Count total video folders across both ZIPs
total_videos = sum(
    sum(os.path.isdir(os.path.join(base, name)) for name in os.listdir(base))
    for base in frame_sources
)

print(f"Total videos to process: {total_videos}")


📊 Total videos to process: 2466


In [ ]:
video_index = 0
for base in frame_sources:
    for folder_name in sorted(os.listdir(base)):
        folder_path = os.path.join(base, folder_name)
        if not os.path.isdir(folder_path):
            continue

        video_index += 1
        print(f"\nProcessing video {video_index}/{total_videos} → {folder_name}")
        frame_paths = sorted([
            os.path.join(folder_path, f)
            for f in os.listdir(folder_path)
            if f.lower().endswith((".jpg", ".png"))
        ])

        if not frame_paths:
            print(" No frames found, skipping.")
            continue

        embeddings = []
        for img_path in frame_paths:
            try:
                embeddings.append(extract_clip_embedding(img_path))
            except Exception as e:
                print(f"Failed on {img_path}: {e}")

        video_embedding = np.mean(embeddings, axis=0) if embeddings else np.zeros(512)
        save_path = os.path.join(output_dir, f"{folder_name}.npy")
        np.save(save_path, video_embedding)
        print(f"Saved embedding: {save_path}")


Streaming output truncated to the last 5000 lines.
🎬 Processing video 800/2466 → 7452069644283694344_frames
✅ Saved embedding: /content/drive/MyDrive/clip_embeddings_all/7452069644283694344_frames.npy

🎬 Processing video 801/2466 → 7452077725981969671_frames
✅ Saved embedding: /content/drive/MyDrive/clip_embeddings_all/7452077725981969671_frames.npy

🎬 Processing video 802/2466 → 7452090912005573904_frames
✅ Saved embedding: /content/drive/MyDrive/clip_embeddings_all/7452090912005573904_frames.npy

🎬 Processing video 803/2466 → 7452092930694614279_frames
✅ Saved embedding: /content/drive/MyDrive/clip_embeddings_all/7452092930694614279_frames.npy

🎬 Processing video 804/2466 → 7452173554621107461_frames
✅ Saved embedding: /content/drive/MyDrive/clip_embeddings_all/7452173554621107461_frames.npy

🎬 Processing video 805/2466 → 7452221712185232647_frames
✅ Saved embedding: /content/drive/MyDrive/clip_embeddings_all/7452221712185232647_frames.npy

🎬 Processing video 806/2466 → 7452244699005